In [38]:
import pandas as pd
import requests

In [42]:
headers = {"X-API-Key": "LFO5p_tz6ZR-zO80_hjXyG-3iDs3LAzdyLASd_ve3m8"}

In [43]:
# Premiere League ID is 9
response = requests.get("https://fbrapi.com/league-seasons", params={"league_id": 9}, headers=headers)
data = response.json()
records = data['data']
pl_seasons = pd.json_normalize(records)
pl_seasons

,season_id,competition_name,#_squads,champion,top_scorer.player,top_scorer.goals_scored
0,2025-2026,Premier League,20,,Erling Haaland,9.0
1,2024-2025,Premier League,20,Liverpool,Mohamed Salah,29.0
2,2023-2024,Premier League,20,Manchester City,Erling Haaland,27.0
3,2022-2023,Premier League,20,Manchester City,Erling Haaland,36.0
4,2021-2022,Premier League,20,Manchester City,"[Son Heung min, Mohamed Salah]",23.0
...,...,...,...,...,...,...
122,1892-1893,Football League First Division,16,Sunderland,,NaN
123,1891-1892,Football League,14,Sunderland,,NaN
124,1890-1891,Football League,12,Everton,,NaN
125,1889-1890,Football League,12,Preston North End,,NaN


In [ ]:
response = requests.get("https://fbrapi.com/matches", params={"league_id": 9}, headers=headers)
data = response.json()
records = data['data']
pl_matches = pd.json_normalize(records)

In [53]:
pl_matches['date'] = pd.to_datetime(pl_matches['date'])
pl_matches = pl_matches[pl_matches['date'] <= pd.Timestamp.now()].sort_values(by='date', ascending=False)
pl_matches

,match_id,date,time,wk,home,home_team_id,away,away_team_id,home_team_score,away_team_score,venue,attendance,referee
69,8c0ee359,2025-10-05,16:30,7,Brentford,cd051869,Manchester City,b8fd03ef,None,None,Gtech Community Stadium,"17,028",Darren England
68,e33f1657,2025-10-05,14:00,7,Newcastle Utd,b2b47a98,Nott'ham Forest,e4a775cb,None,None,St James' Park,"52,189",Peter Bankes
67,db870461,2025-10-05,14:00,7,Aston Villa,8602292d,Burnley,943e8050,None,None,Villa Park,"40,308",Andrew Kitchen
66,88e53add,2025-10-05,14:00,7,Everton,d3fd31cc,Crystal Palace,47c64c55,None,None,Hill Dickinson Stadium,"51,770",Michael Salisbury
65,1314f9fd,2025-10-05,14:00,7,Wolves,8cec06e1,Brighton,d07537b9,None,None,Molineux Stadium,"29,504",Jarred Gillett
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,e1058522,2025-08-16,15:00,1,Sunderland,8ef52968,West Ham,7c21e445,None,None,Stadium of Light,"46,233",Robert Jones
1,bbdf4739,2025-08-16,12:30,1,Aston Villa,8602292d,Newcastle Utd,b2b47a98,None,None,Villa Park,"42,526",Craig Pawson
4,d41bb8b3,2025-08-16,15:00,1,Tottenham,361ca564,Burnley,943e8050,None,None,Tottenham Hotspur Stadium,"61,077",Michael Oliver
5,8613020d,2025-08-16,17:30,1,Wolves,8cec06e1,Manchester City,b8fd03ef,None,None,Molineux Stadium,"31,456",Jarred Gillett


In [67]:
tms = pl_matches['home'].unique()
ids = pl_matches['home_team_id'].unique()

teams = {}
for i in range(len(tms)):
    teams[tms[i]] = ids[i]
print(teams)

{'Brentford': 'cd051869', 'Newcastle Utd': 'b2b47a98', 'Aston Villa': '8602292d', 'Everton': 'd3fd31cc', 'Wolves': '8cec06e1', 'Chelsea': 'cff3d9bb', 'Manchester Utd': '19538871', 'Arsenal': '18bb7c10', 'Leeds United': '5bfb9659', 'Bournemouth': '4ba7cbea', 'Tottenham': '361ca564', 'Crystal Palace': '47c64c55', 'Manchester City': 'b8fd03ef', "Nott'ham Forest": 'e4a775cb', 'Sunderland': '8ef52968', 'Fulham': 'fd962109', 'West Ham': '7c21e445', 'Burnley': '943e8050', 'Brighton': 'd07537b9', 'Liverpool': '822bd0ba'}


In [88]:
import time
import random

pl_matches_2 = pd.DataFrame()
for team in teams.keys():
    response = requests.get("https://fbrapi.com/teams", params={"team_id": teams[team]}, headers=headers)
    data = response.json()
    records = data['team_schedule']['data']
    df = pd.json_normalize(records)
    df['home_team'] = team
    df['home_team_id'] = teams[team]
    pl_matches_2 = pd.concat([pl_matches_2, df])
    time.sleep(random.uniform(1, 3))

In [89]:
pl_matches_2['date'] = pd.to_datetime(pl_matches_2['date'])
pl_matches_2 = pl_matches_2[pl_matches_2['date'] <= pd.Timestamp.now()].sort_values(by='date', ascending=False)
pl_matches_2

,date,time,match_id,league_name,league_id,opponent,opponent_id,home_away,result,gf,ga,attendance,captain,formation,referee,home_team,home_team_id
9,2025-10-05,14:00,db870461,Premier League,9,Burnley,943e8050,Home,W,2.0,1.0,"40,308",John McGinn,4-2-2-2,Andrew Kitchen,Aston Villa,8602292d
11,2025-10-05,14:00,88e53add,Premier League,9,Everton,d3fd31cc,Away,L,1.0,2.0,"51,770",Marc Guéhi,3-4-3,Michael Salisbury,Crystal Palace,47c64c55
8,2025-10-05,14:00,1314f9fd,Premier League,9,Wolves,8cec06e1,Away,D,1.0,1.0,"29,504",Lewis Dunk,4-2-3-1,Jarred Gillett,Brighton,d07537b9
8,2025-10-05,14:00,88e53add,Premier League,9,Crystal Palace,47c64c55,Home,W,2.0,1.0,"51,770",James Tarkowski,4-2-3-1,Michael Salisbury,Everton,d3fd31cc
8,2025-10-05,14:00,db870461,Premier League,9,Aston Villa,8602292d,Away,L,1.0,2.0,"40,308",Josh Cullen,5-4-1,Andrew Kitchen,Burnley,943e8050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2025-08-15,20:00,a071faa8,Premier League,9,Liverpool,822bd0ba,Away,L,2.0,4.0,"60,315",Adam Smith,4-1-4-1,Anthony Taylor,Bournemouth,4ba7cbea
1,2025-08-15,20:00,a071faa8,Premier League,9,Bournemouth,4ba7cbea,Home,W,4.0,2.0,"60,315",Virgil van Dijk,4-2-3-1,Anthony Taylor,Liverpool,822bd0ba
0,2025-08-13,21:00,7eef879c,Super Cup,122,Paris S,e2d8892c,Neutral,D,2.0,2.0,"21,025",Cristian Romero,4-3-3,João Pinheiro,Tottenham,361ca564
0,2025-08-10,15:00,096e63eb,FA Community Shield,602,Crystal Palace,47c64c55,Neutral,D,2.0,2.0,"82,645",Virgil van Dijk,4-2-3-1,Chris Kavanagh,Liverpool,822bd0ba
